This notebook was created on December 2024. 


Its primary purpose is to generate and manipulate small road networks that can serve as inputs for tests.


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import random

print(f"""OSMnx version: {ox.__version__}""")

f"""NetworkX version: {nx.__version__}"""

### What is an OSMnx network?

Copied from [OSMnx 2.0.0 docs](https://osmnx.readthedocs.io/en/stable/getting-started.html#model-attributes): 

As a NetworkX MultiDiGraph object, it has top-level graph, nodes, and edges attributes. 
- The graph attribute dictionary must contain a “crs” key defining its coordinate reference system. 

- The nodes are identified by OSM ID and each must contain a data attribute dictionary that must have “x” and “y” keys defining its coordinates and a “street_count” key defining how many physical streets are incident to it. 

- The edges are identified by a 3-tuple of “u” (source node ID), “v” (target node ID), and “key” (to differentiate parallel edges), and each must contain a data attribute dictionary that must have an “osmid” key defining its OSM ID and a “length” key defining its length in meters.



To create a road network, one can make a NetworkX object with the properties above. 

In this notebook we take a different approach: we download a small road network and modify it as needed.

### Specify place

In [74]:
alameda = {
    "city": "Alameda",
    "county": "Alameda County",
    "state": "California",
    "country": "USA",
}

### Download actual walking road network

In [ ]:
G = ox.graph_from_place(alameda, network_type="walk")

type(G)

In [ ]:
f"""Alameda city's walking road network has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges"""

That's still too large for our purposes. Let's choose a subgraph

In [ ]:
# Fix a seed to get reproducible results
random.seed(42)

# choose a random node
ego_node = random.choice(list(G.nodes))

ego_node

### Getting a subgraph

In [ ]:
# Get the subgraph of G that contains the ego_node and up to 2 nodes away from it. Includes edges

subgraph = nx.ego_graph(G, ego_node, radius=2)

f"""Subgraph has {subgraph.number_of_nodes()} nodes and {subgraph.number_of_edges()} edges"""

In [ ]:
subgraph.nodes[ego_node]

### Here's a pretty picture

Ok, a picture

In [ ]:
nx.draw_networkx(subgraph)

# Set margins for the axes so nodes aren't clipped
ax = plt.gca()
ax.margins(0.20)
plt.axis("off")
plt.show()


# notice that the ego_node is NOT the one in the center

### Save/load network as a graphml file

In [81]:
# This notebook is in the test_data folder because it makes logical
# sense as it is used to create test_data. Plus, it simplifies the filepath

filepath = "subgraph.graphml"

ox.save_graphml(subgraph, filepath)

Did that work? Let's load it

In [82]:
subgraph_saved = ox.load_graphml(filepath)

# this function exists in nx 3.4.2:
# nx.graphs_equal(subgraph, subgraph_saved)

Since we can't use graphs_equal until we update nx, let's do some checks:

In [ ]:
subgraph.nodes(data=True) == subgraph_saved.nodes(data=True)

In [ ]:
list(subgraph.edges(data=True)) == list(subgraph_saved.edges(data=True))

### Yay!

Ok, sorry if it didn't work for you... but I'm celebrating because it works for me now :)

### Finally: please keep test_data clean :)

- If you end up using the graph in a test, give it a sensible name (not subgraph, like I did here)
- If you don't end up using the graph, please remove it:

In [ ]:
from pathlib import Path

file_path = Path(filepath)
if file_path.exists():
    file_path.unlink()
    print(f"{file_path} has been removed.")
else:
    print(f"{file_path} does not exist.")

## Adding speed and travel time attributes to edges.
> like it is done in layers.py (get_road_network() method

In [ ]:
# The edges from this subgraph have a length attribute but no speed attribute
# subgraph.edges(data=True)
print(subgraph.edges(data="length"))
# let's add a speed attribute
default_speed_walking = 4
# nx.set_edge_attributes(subgraph, default_speed_walking, "speed_kph")
ox.add_edge_speeds(subgraph, hwy_speeds={"footway": 4, "driving": 50, "bike": 15})
print(subgraph.edges(data="speed_kph"))
# now that it has a speed attribute we can add a travel time attribute
ox.add_edge_travel_times(subgraph)
print(subgraph.edges(data="travel_time"))
time = nx.get_edge_attributes(subgraph, "travel_time")
print(time)
time_min = dict(zip(list(time.keys()), list(map(lambda x: round(x / 60, 2), time.values()))))
print(time_min)
nx.set_edge_attributes(subgraph, time_min, "travel_time")
print(subgraph.edges(data=True))